SmartStock – AI-Powered Inventory Rebalancing for Walmart

Step 1: Simulating the Retail World (Dataset Creation)

This notebook simulates a miniature version of Walmart's store inventory system for use in our hackathon project: **SmartStock**.

We generate:
- 10 stores
- 10 products (SKUs)
- 7 days of data
- Weather conditions affecting demand (e.g., rainy = more umbrellas sold)

Each row in the dataset contains:
- Day
- Store ID
- Product ID
- Weather
- Inventory available
- Sales on that day

This synthetic data will power our later stages: **demand forecasting**, **inventory balancing**, and **visualization**.

---


In [5]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import random

# Setup
num_stores = 10
num_products = 10
days = 7
store_ids = [f"Store_{i+1}" for i in range(num_stores)]
product_ids = [f"SKU_{j+1}" for j in range(num_products)]
weather_conditions = ["sunny", "rainy", "cloudy"]

# Data generation
data = []

for day in range(1, days + 1):
    for store in store_ids:
        for product in product_ids:
            weather = random.choice(weather_conditions)
            base_demand = np.random.randint(5, 20)
            # Increase demand if rainy for rain-sensitive products
            if weather == "rainy" and product in ["SKU_1", "SKU_2"]:
                demand = int(base_demand * 1.8)
            else:
                demand = base_demand

            inventory = np.random.randint(10, 50)
            sales = min(demand, inventory)

            data.append({
                "day": day,
                "store_id": store,
                "product_id": product,
                "weather": weather,
                "inventory": inventory,
                "sales": sales
            })

# Convert to DataFrame
df = pd.DataFrame(data)

# Preview the data
df.head()


,day,store_id,product_id,weather,inventory,sales
0,1,Store_1,SKU_1,cloudy,31,9
1,1,Store_1,SKU_2,rainy,31,16
2,1,Store_1,SKU_3,rainy,18,6
3,1,Store_1,SKU_4,cloudy,10,10
4,1,Store_1,SKU_5,cloudy,42,14


In [6]:
# Forecast tomorrow's demand using average of last 3 days (simplified)
def forecast_demand(df):
    forecast = (
        df.groupby(["store_id", "product_id"])["sales"]
        .rolling(window=3, min_periods=1)
        .mean()
        .shift(
            1)
        .reset_index()
        .rename(columns={"sales": "forecasted_demand"})
    )
    df = df.merge(forecast, on=["level_0"], how="left")
    return df


In [9]:
#OHE on weather 
df['day_of_week'] = df['day'] % 7

# Rolling 3-day average sales per store-product
df['past_3_avg'] = (
    df.groupby(['store_id', 'product_id'])['sales']
    .transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
)

# One-hot encode weather
df = pd.get_dummies(df, columns=['weather'])


In [14]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [18]:
# Step 1: Feature Engineering
df['day_of_week'] = df['day'] % 7
df['past_3_avg'] = (
    df.groupby(['store_id', 'product_id'])['sales']
    .transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
)

# Drop rows with missing values (from rolling avg)
df = df.dropna()

# Step 2: Prepare features and target
features = ['inventory', 'past_3_avg', 'day_of_week',
            'weather_cloudy', 'weather_rainy', 'weather_sunny']
X = df[features]
y = df['sales']

# Step 3: Split and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(objective="reg:squarederror", n_estimators=100)
model.fit(X_train, y_train)

# Step 4: Predict demand
df['forecasted_demand'] = model.predict(X)

# Step 5: Evaluate
rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)
print(f"XGBoost trained | RMSE = {rmse:.2f}")


XGBoost trained | RMSE = 5.24


C:\Users\ankit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [24]:
def suggest_transfers(df, threshold=5):
    transfers = []

    # Use only the last day for transfer decisions
    latest_day = df['day'].max()
    df_day = df[df['day'] == latest_day]

    grouped = df_day.groupby("product_id")

    for product, group in grouped:
        understocked = group[group['inventory'] + threshold < group['forecasted_demand']]
        overstocked = group[group['inventory'] - threshold > group['forecasted_demand']]

        # 🔍 Debug prints
        print(f"\n Product: {product}")
        print("Understocked Stores:")
        print(understocked[['store_id', 'inventory', 'forecasted_demand']])
        print("Overstocked Stores:")
        print(overstocked[['store_id', 'inventory', 'forecasted_demand']])

        for _, need_row in understocked.iterrows():
            for _, surplus_row in overstocked.iterrows():
                if need_row['store_id'] == surplus_row['store_id']:
                    continue  # skip same store

                # Transfer logic
                qty_needed = need_row['forecasted_demand'] - need_row['inventory']
                qty_available = surplus_row['inventory'] - surplus_row['forecasted_demand']
                transfer_qty = int(min(qty_needed, qty_available))

                if transfer_qty > 0:
                    transfers.append({
                        "product": product,
                        "from_store": surplus_row['store_id'],
                        "to_store": need_row['store_id'],
                        "quantity": transfer_qty
                    })

    return pd.DataFrame(transfers)


In [25]:
transfers_df = suggest_transfers(df)
print("Suggested Transfers:\n", transfers_df.head())


 Product: SKU_1
Understocked Stores:
Empty DataFrame
Columns: [store_id, inventory, forecasted_demand]
Index: []
Overstocked Stores:
    store_id  inventory  forecasted_demand
600  Store_1         40          24.873100
610  Store_2         44          17.970655
640  Store_5         26          16.844440
660  Store_7         25          17.034859
680  Store_9         35          11.105490

 Product: SKU_10
Understocked Stores:
Empty DataFrame
Columns: [store_id, inventory, forecasted_demand]
Index: []
Overstocked Stores:
    store_id  inventory  forecasted_demand
619  Store_2         38          16.986290
639  Store_4         37          19.347809
659  Store_6         42          13.948621
669  Store_7         32          16.597277
679  Store_8         30          12.962046
689  Store_9         39           7.558304

 Product: SKU_2
Understocked Stores:
Empty DataFrame
Columns: [store_id, inventory, forecasted_demand]
Index: []
Overstocked Stores:
     store_id  inventory  forecasted_d

In [26]:
latest_day = df['day'].max()

# Force Store_1 to be understocked in SKU_1
df.loc[(df['store_id'] == 'Store_1') & (df['product_id'] == 'SKU_1') & (df['day'] == latest_day), 'inventory'] = 3

# Force Store_4 to be understocked in SKU_2
df.loc[(df['store_id'] == 'Store_4') & (df['product_id'] == 'SKU_2') & (df['day'] == latest_day), 'inventory'] = 2

In [27]:
transfers_df = suggest_transfers(df, threshold=2)
print(transfers_df)


 Product: SKU_1
Understocked Stores:
    store_id  inventory  forecasted_demand
600  Store_1          3            24.8731
Overstocked Stores:
    store_id  inventory  forecasted_demand
610  Store_2         44          17.970655
620  Store_3         12           7.022234
640  Store_5         26          16.844440
650  Store_6         12           8.574615
660  Store_7         25          17.034859
680  Store_9         35          11.105490

 Product: SKU_10
Understocked Stores:
Empty DataFrame
Columns: [store_id, inventory, forecasted_demand]
Index: []
Overstocked Stores:
     store_id  inventory  forecasted_demand
619   Store_2         38          16.986290
629   Store_3         13          10.131487
639   Store_4         37          19.347809
659   Store_6         42          13.948621
669   Store_7         32          16.597277
679   Store_8         30          12.962046
689   Store_9         39           7.558304
699  Store_10         13          10.002428

 Product: SKU_2
Underst

In [28]:
# Artificially spike forecasted demand to simulate panic buying or local promo
df.loc[(df['store_id'] == 'Store_1') & (df['product_id'] == 'SKU_1') & (df['day'] == latest_day), 'forecasted_demand'] = 35

In [29]:
transfers_df = suggest_transfers(df, threshold=2)
print(transfers_df)


 Product: SKU_1
Understocked Stores:
    store_id  inventory  forecasted_demand
600  Store_1          3               35.0
Overstocked Stores:
    store_id  inventory  forecasted_demand
610  Store_2         44          17.970655
620  Store_3         12           7.022234
640  Store_5         26          16.844440
650  Store_6         12           8.574615
660  Store_7         25          17.034859
680  Store_9         35          11.105490

 Product: SKU_10
Understocked Stores:
Empty DataFrame
Columns: [store_id, inventory, forecasted_demand]
Index: []
Overstocked Stores:
     store_id  inventory  forecasted_demand
619   Store_2         38          16.986290
629   Store_3         13          10.131487
639   Store_4         37          19.347809
659   Store_6         42          13.948621
669   Store_7         32          16.597277
679   Store_8         30          12.962046
689   Store_9         39           7.558304
699  Store_10         13          10.002428

 Product: SKU_2
Underst

In [30]:
def compute_availability(df, transfers_df):
    latest_day = df['day'].max()
    df_day = df[df['day'] == latest_day].copy()

    df_day['available'] = df_day['inventory'] >= df_day['forecasted_demand']
    before = df_day['available'].mean() * 100

    # Apply transfers (simulate inventory changes)
    for _, row in transfers_df.iterrows():
        from_mask = (df_day['store_id'] == row['from_store']) & (df_day['product_id'] == row['product'])
        to_mask = (df_day['store_id'] == row['to_store']) & (df_day['product_id'] == row['product'])

        df_day.loc[from_mask, 'inventory'] -= row['quantity']
        df_day.loc[to_mask, 'inventory'] += row['quantity']

    df_day['available'] = df_day['inventory'] >= df_day['forecasted_demand']
    after = df_day['available'].mean() * 100

    print(f"\n Availability Before Transfers: {before:.2f}%")
    print(f"Availability After Transfers : {after:.2f}%")

    return before, after

In [31]:
compute_availability(df, transfers_df)


 Availability Before Transfers: 97.00%
Availability After Transfers : 99.00%


(97.0, 99.0)

That 2% increase in product availability can mean millions in revenue retention at Walmart’s scale. And it’s achieved with:
1.No extra purchases
2.Just smart redistribution

In [33]:
# Save main data
df.to_csv("your_final_dataframe.csv", index=False)

# Save transfer suggestions
transfers_df.to_csv("transfer_output.csv", index=False)